#PREPROCESSING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

importing nessary modules

In [ ]:
import pandas as pd
import numpy as np
import os
import csv
import json
import time

preprocessing the flight data

In [ ]:
base_path = '/content/drive/My Drive/flight project /flight'
airport_list = ['ATL', 'CLT', 'DEN', 'DFW', 'EWR', 
                'IAH', 'JFK', 'LAS', 'LAX', 'MCO',
                'MIA', 'ORD', 'PHX', 'SEA', 'SFO']
temp = pd.DataFrame()
flight_df = pd.DataFrame()
for filename in os.listdir(base_path):
    temp = pd.read_csv(base_path + '/' + filename + '/' + filename + '.csv')
    temp = temp.loc[:, ['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime', 
                             'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'OriginAirportID', 
                             'DestAirportID', 'ArrTime', 'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes',
                             'Origin', 'Dest'] ]
    temp = temp[temp['CRSDepTime'].notnull()]
    temp['mod_dep_time'] = temp['CRSDepTime'].apply(lambda x: (((x // 100) + ((x % 100) // 30)) * 100) if x // 100 != 23 
                                                 else ((x // 100) * 100) )
    flight_df = pd.concat([flight_df, temp])
flight_df = flight_df[flight_df['Origin'].isin(airport_list) & flight_df['Dest'].isin(airport_list)]
flight_df.to_csv('/content/drive/My Drive/flight project /flightdata.csv',index = False)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/p

preprocessing the weather data

In [ ]:
weather_df = pd.DataFrame()
base_path =os.getcwd()
year = ['2016','2017']
airport_list = ['ATL', 'CLT', 'DEN', 'DFW', 'EWR',
                'IAH', 'JFK', 'LAS', 'LAX', 'MCO',
                'MIA', 'ORD', 'PHX', 'SEA', 'SFO']
for j in airport_list:
    for k in os.listdir('/content/drive/My Drive/flight project /weather/' + j):
      if(k[0:4] in year):
        df = pd.read_json('/content/drive/My Drive/flight project /weather/' + j + '/' + k)
        for i in range(len(df.data.weather)):
            temp = pd.DataFrame(df.data.weather[i]['hourly'])
            temp['date'] = df.data.weather[i]['date']
            temp['airport'] = j
            weather_df = pd.concat([weather_df, temp])
weather_df = weather_df.loc[ : , ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',                               
                                 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 
                                 'tempF', 'WindChillF', 'humidity', 'date', 'time', 'airport']] 
weather_df.to_csv('/content/drive/My Drive/flight project ' + '/weatherdata.csv',index = False)


merging flight and airport and getting final dataset

In [ ]:
final_df = pd.merge(flight_df, weather_df,  how='inner', left_on=['Origin', 'mod_dep_time', 'FlightDate'], right_on = ['airport', 'time', 'date'])
final_df.dropna()
final_df.to_csv('/content/drive/My Drive/flight project ' + '/finaldata.csv', index = False)
